# Ice Cream Shops Exploration

## Business Problem

A young, international and fast growing chain of Ice Cream Shops would like to expand their business to Germany. The business owners are not familiar with current situation of Ice Cream Business in Germany and they would like to explore it better. So they would like to collect statistics about current presence of ice cream shops in German cities. 

Also, the company would like to receive the list of best candidates cities to open new shops, based on few criteria:
- the cities with the smallest density of existing coffee shops per 1000 person of population;
- the cities with at least 10000 people of population

## Data

In order to collect the required statistics about the Coffee Shops in German cities, we need: 
1. The list of german cities with population data for each; 
2. The information about current Ice Cream Shops for each city. 

__The list of german cities with corresponding population__ was found on the web page of Statistisches Bundesamt: https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/05-staedte.html). Here, the corresponding .csv file was downloaded and will be used from local repository. This file contains list of all locations which have status of city, along with Postal Code and Population data. 

Additionally, since the data file from Statistisches Bundesamt has only one Postal Code per city, but the large cities as Berlin or Hamburg have many Postal Codes, additional database with __all__ postal codes as well as with latitude and longitude coordinates was downloaded from https://github.com/zauberware/postal-codes-json-xml-csv for Germany.

__The information about current Ice Cream Shops__ will be retrieved from ForeSquares.com API based on coordinates of the cities from above databases. 

